In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import py.continuum1d as continuum1d

from py.analysis import computeFPs, checkFixPtsStability

from py.funcs import getAvgPSD, getPSD
from py.params import setParams

# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2


In [2]:
df = pd.read_csv('csv/adap_b05_taua1000.csv')

df_compute = df[df['p_down']==3]

In [3]:
params=None
c1d = continuum1d.continuum1d()

1d Model initialised.


In [4]:
a = df_compute #df_show[df_show['p_down']==3]
print(len(a))
ext_values = np.zeros((len(a), 2))

count = 0
for index, row in a.iterrows():
   # print(row)
    ext_values[count][0] = row['I_e']
    ext_values[count][1] = row['I_i']
    count+=1

ext_values

df_columns=['I_e', 'I_i', 'dom_frequ', 'up_space', 'down_space']
df_new = pd.DataFrame(columns=df_columns)

count = 0
for array in ext_values:
    ee = array[0]
    ii = array[1]
    print(count, ee, ii)
    count+=1
    
    params={'I_e': ee, 'I_i': ii, 'b': 0.5, 'tau_a': 1000, 'end_t': 10*1000}
    ps = setParams(params)
    
    
    fps = computeFPs(ps)
    if not fps.size>0:
        fp = np.array([0.0, 0.01])
    else:
        fp=fps[0]
    stab = checkFixPtsStability(fps, ps)
    
    exc, inh = c1d.run(ps, itype='inte_fft', fp = fp)
    
    exc_test = exc.T[:,-10000]
    
 #   print(exc_test)
    
    frequs, PSD = getPSD(exc_test, ps.n)

    
    dom_frequ = frequs[np.argmax(PSD)]    
    
    if count in [0,2,4,6,8]:
        plt.figure(figsize=(3,3))
        
        plt.imshow(exc.T[:,-500:], vmin=0, vmax=1, aspect='auto')
        plt.show()
        
        plt.figure(figsize=(3,3))
        plt.plot(frequs, PSD)
        plt.show()
        print(dom_frequ)
    
    
    e = exc_test
    maxi = max(e)
    up_nodes = sum(x>=0.4*maxi for x in e)
    down_nodes = sum(x < 0.4*maxi for x in e)
    up_nodes, down_nodes
    nodes_per_down = down_nodes/dom_frequ
    nodes_per_up = up_nodes/dom_frequ
    space_per_down = ps.dx * nodes_per_down
    space_per_up = ps.dx * nodes_per_up

    values = [[ee, ii, dom_frequ, space_per_up, space_per_down]]
    df_temp = pd.DataFrame(values, columns=df_columns)
    df_new = pd.concat([df_new, df_temp])
    
    
    
df_new

1
0 0.4333333333333333 -0.1999999999999999


/Users/ronja/opt/anaconda3/lib/python3.9/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 8192 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/var/folders/8l/w8cdnkkj38bg2pscx__1vxth0000gn/T/ipykernel_3238/1289211264.py:63: RuntimeWarning: invalid value encountered in true_divide
  nodes_per_down = down_nodes/dom_frequ
/var/folders/8l/w8cdnkkj38bg2pscx__1vxth0000gn/T/ipykernel_3238/1289211264.py:64: RuntimeWarning: divide by zero encountered in true_divide
  nodes_per_up = up_nodes/dom_frequ


,I_e,I_i,dom_frequ,up_space,down_space
0,0.433333,-0.2,0.0,inf,NaN


In [5]:
df_new.to_csv('csv/adaps_space_frequs.csv')